In [1]:
from smolagents import CodeAgent, LiteLLMModel
from webofagents.core.tools import AgentSearch, AskAgent

model = LiteLLMModel(
    model_id="ollama_chat/llama3.3",
    api_base="http://localhost:11434",
    num_ctx=8192) # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.

# -----------------------------
# Define the expert agents
# -----------------------------
medical_expert_agent = CodeAgent(tools=[], model=model, add_base_tools=True)
cs_expert_agent = CodeAgent(tools=[], model=model, add_base_tools=True)

# additional_instructions = """\nYou are an expert in the medical domain and others will ask you questions of your domain. However, they do not know how to structure their queries properly. When you receive a '<<hello>>' token as message, you should respond with a documentation of your expertise domain, what type of queries you are expecting, and how they should structure their queries for the best response."""

# -----------------------------
# Create a registry mapping expert names to domain keywords
# -----------------------------
expert_registry = {
    "medical_expert": ["medical", "health", "cancer", "disease", "immunotherapy", "medicine", "headache", "dizziness", "heart", "cardiac", "cardiologist", "doctor"],
    "cs_expert": ["computer", "software", "programming", "algorithm", "hardware", "data"]
}

# Instantiate AgentSearch with the registry
agent_search_tool = AgentSearch(registry=expert_registry)

# Instantiate AskAgent with a mapping from expert names to expert agents
ask_agent_tool = AskAgent(expert_dict={
    "medical_expert": medical_expert_agent,
    "cs_expert": cs_expert_agent
})



/home/risharma/.conda/envs/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from smolagents import Tool

class MedicalReportReader(Tool):
    name = "medical_report_reader"
    description = """
    Reads the patient's medical report from a text file given the file path. If there is a medical query, this tool must be used to fetch the patient's medical report.
    The expert agent will be able to provide a better answer if the medical report is provided in the query.
    The medical report can help provide a better answer.
    The patient record is already initialized and in memory, so you do not need to pass any parameters.
    The record is returned as a string containing all the patient's medical information, history, vitals, etc.
    """
    inputs = {
        # "path": {
        #     "type": "string",
        #     "description": "The file path of the patient's medical report text file."
        # }
    }
    output_type = "string"
    def __init__(self, filepath: str, *args, **kwargs):
        self.filepath = filepath
        super().__init__(*args, **kwargs)

    def forward(self):
        try:
            with open(self.filepath, "r") as file:
                content = str(file.read())
            return content
        except Exception as e:
            return f"Error reading file: {e}"

medical_report_reader = MedicalReportReader("/mnt/nfs/home/risharma/patient-1.txt")

In [3]:
# -----------------------------------------------------
# Create Personal Agent A with the Search and Ask tools
# -----------------------------------------------------
agent_a = CodeAgent(
    tools=[agent_search_tool, ask_agent_tool, medical_report_reader],
    model=model,
    add_base_tools=False
)

medical_question = "I feel dizzy and have a headache. Look at my Patient Medical Report (history) using the tools provided and let me know what should I do? If I need to see a doctor, suggest me a good one near MIT Media Lab."
final_answer_medical = agent_a.run(medical_question)
print("Final Answer (Medical):", final_answer_medical)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I feel dizzy and have a headache. Look at my Patient Medical Report (history) using the tools provided and let  │
│ me know what should I do? If I need to see a doctor, suggest me a good one near MIT Media Lab.                  │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.3 ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  medical_report = medical_report_reader()                                                                         
  print(medical_report)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Patient Medical Report #1

Patient Name: John Doe
Age: 45
Gender: Male
Chief Complaint: Chest pain and shortness of breath occurring intermittently over the past two days, exacerbated by
physical exertion and relieved by rest. Episodes last approximately 10-15 minutes.
Medical History: Hypertension diagnosed 5 years ago, occasional smoker (smoked 5-7 cigarettes daily, quit 2 years 
ago), family history significant for cardiovascular disease (father had myocardial infarction at age 50).
Current Medications: Lisinopril 10 mg once daily, Aspirin 81 mg daily (started 6 months ago)
Vital Signs: BP: 150/95 mmHg, HR: 92 bpm, Temp: 98.6°F, Respiratory Rate: 18 breaths/min, SpO2: 95% on room air.
Physical Examination: Mildly overweight (BMI: 28.5 kg/m²), anxious appearance, clear lungs bilaterally, regular 
heart rhythm with no murmurs or gallops, abdomen soft and non-tender, extremities warm with no peripheral edema or 
cyanosis.
Lab Results: LDL cholesterol markedly elevated at 190 mg/dL, HDL cholesterol at 35 mg/dL, triglycerides 180 mg/dL, 
mildly elevated troponin-I at 0.05 ng/mL (normal <0.04 ng/mL). Normal CBC, electrolytes, and renal function 
tests.Imaging Results: ECG reveals ST-segment depression in leads II, III, and aVF indicative of myocardial 
ischemia. Chest X-ray is normal, no evidence of cardiomegaly or pulmonary congestion.


Out: None

[Step 1: Duration 5.91 seconds| Input tokens: 2,636 | Output tokens: 68]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The patient should seek medical attention immediately. Some highly-rated cardiologists near MIT    
  Media Lab in Cambridge, MA include Dr. Jeffrey Leavitt, Dr. Leonard Zir, and Dr. Thomas Ohalloran.")             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The patient should seek medical attention immediately. Some highly-rated cardiologists near MIT
Media Lab in Cambridge, MA include Dr. Jeffrey Leavitt, Dr. Leonard Zir, and Dr. Thomas Ohalloran.

[Step 5: Duration 11.35 seconds| Input tokens: 21,257 | Output tokens: 601]

Final Answer (Medical): The patient should seek medical attention immediately. Some highly-rated cardiologists near MIT Media Lab in Cambridge, MA include Dr. Jeffrey Leavitt, Dr. Leonard Zir, and Dr. Thomas Ohalloran.
